In [ ]:
import warnings
warnings.filterwarnings("ignore") #used to stop unwanted warnings from showing when you execute codes

#import necessary libraries 
#pandas for data manipulation and analysis
#matplotlib and seaborn for visualization
#numpy for manipulation of numeric data adn arrays

import pandas as pd, matplotlib.pyplot as plt, numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical

In [ ]:
#import and read the dataset
data = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')
data.head() #shows only first 5 rows by default

In [ ]:
data.isnull().sum() #checking for null values in the dataset

In [ ]:
data.duplicated().sum() #checking for duplicated values in the dataset

In [ ]:
data.info() #information about the data

In [ ]:
data.describe() #returns a descriptive statistics summary 

In [ ]:
cols = ["RowNumber", "CustomerId", "Surname"]
data = data.drop(columns = cols , axis=1)

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(12, 5))
data.boxplot()
plt.show()

In [ ]:
#splitting data into target and features
x= data.iloc[: , :-1] #features
y=data.iloc[:, -1] #target

In [ ]:
x.head()

In [ ]:
y.head()

### Removing outliers

In [ ]:
x.skew(numeric_only=None)

if skew < 0.5 => there are no outliers => normal distribution

we can notice that the variables "Age"  have many outliers 

In [ ]:
x.boxplot(column=['Age'], return_type='axes');

In [ ]:
from scipy.stats import norm
sns.distplot(data.Age, fit = norm) 
#displays the plot of the dataset compared to the normal distribution

In [ ]:
age_log = np.log(data.Age) #applying log function to the dataset

In [ ]:
sns.distplot(age_log, fit = norm) 

### Converting categorical variables:

In [ ]:
data.select_dtypes(exclude='number')

#### Transform column : gender

In [ ]:
ohe = OneHotEncoder() #initialize one hot encoder object
transformed = ohe.fit_transform(x[['Gender']]) #fit transform the feature "state"
#print(transformed.toarray())

In [ ]:
print(ohe.categories_) #print categorical values of the features "state"

In [ ]:
x[ohe.categories_[0]] = transformed.toarray() 
#convert the transformed categorical variables into a numpy array and add them to the dataset x
x.head()

#### Transform column Geography:

In [ ]:
data.Geography.unique()

In [ ]:
ohe1 = OneHotEncoder() #initialize one hot encoder object
transformed1 = ohe1.fit_transform(x[['Geography']]) #fit transform the feature "state"
#print(transformed1.toarray())

In [ ]:
print(ohe1.categories_) #print categorical values of the features "state"

In [ ]:
x[ohe1.categories_[0]] = transformed1.toarray() 
#convert the transformed categorical variables into a numpy array and add them to the dataset x
x.head()

In [ ]:
x = x.drop("Spain", axis=1)
x.head()

In [ ]:
cols=["Geography", "Gender", "Male"]
x = x.drop(columns= cols, axis=1)
x.head()

In [ ]:
x.shape

In [ ]:
#split the two datasets x and y into test and training sets
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)
#test_size = 0.2 : 80% training 20% testing

### Standard Scaler

In [ ]:
scaler = StandardScaler() #initialize the standard scaler object

cols=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary'] #choose columns on which you want to apply scaling
X_train[cols]=scaler.fit_transform(X_train[cols]) #fit transform the columns chosen in the training set
X_test[cols]=scaler.transform(X_test[cols]) #fit transform the columns chosen in the testing set

In [ ]:
X_train.head()

In [ ]:
X_train.shape

### Implementing our first Deep learning model :

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(6, input_shape=(11,))) #6 = (sum(input variables+target variables)/2)
model.add(Activation('sigmoid'))

model.add(Dense(6))
model.add(Activation('sigmoid'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
hist = model.fit(X_train, y_train, 
                batch_size= 10,
                epochs= 100, 
                verbose=2,
                validation_split= 0.2)
#we have 10000 => 8000 train + 1000 test => 0.2 validation 
# => 1600 of training set  => we have 6400 training set remaining  => 6400= 640/epoche

val_loss> loss value : change activation function

In [ ]:
model.summary()

dense : param = 11(input v.) *6 = 6(layer_0)*6(layer_1) +6(layer_1) =72

dense_1 : param = 6(layer_1)*6(layer_2) + 6(layer_2) =42


## Changing the activation function:

In [ ]:
model.add(Dense(6, input_shape=(11,))) #6 = (sum(input variables+target variables)/2)
model.add(Activation('relu'))

model.add(Dense(6))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

hist = model.fit(X_train, y_train, 
                batch_size= 10,
                epochs= 100, 
                verbose=2,
                validation_split= 0.2)

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
#y_test = y_test.values

In [ ]:
for i in range(10):
    print(y_test[i], y_pred[i])

In [ ]:
"""
confusion matrix
"""

### Changing parameters of Adam Optimizer

In [ ]:
opt = keras.optimizers.legacy.Adam(learning_rate=0.003, decay=  0.0001)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)

In [ ]:
hist = model.fit(X_train, y_train, 
                batch_size= 10,
                epochs= 100, 
                verbose=2,
                validation_split= 0.2)

In [ ]:
#create model , + gridsearch : parameters : batch size [10,20] epoche [100 120] 

In [ ]:
def create_model():
    model.add(Dense(6, input_shape=(11,))) #6 = (sum(input variables+target variables)/2)
    model.add(Activation('relu'))

    model.add(Dense(6))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    opt = keras.optimizers.legacy.Adam(learning_rate=0.003, decay=  0.0001)
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)
    
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_tn = create_model)
parameters = {'batch_size':[10,20],
             'epochs':[100,120]}
grid_search = GridSearchCV(estimator = model, 
                           param_grid = parameters, 
                           scoring='accuracy',
                           cv=10)
grid_search= grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_